In [ ]:
import re
import sys
import pathlib
import functools
import traceback

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import IPython.display

import pydicom

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import pymedphys

In [ ]:
percent_deviation = 2
mm_dist_threshold = 0.5

logfile_directory = pathlib.Path(r'D:\LinacLogFiles')
monaco_directory = pathlib.Path(r'\\monacoda\FocalData\RCCC\1~Clinical')
output_directory = pathlib.Path(r'S:\Physics\Patient Specific Logfile Fluence')

In [ ]:
def get_incomplete_qcls(cursor, location):
    data = pymedphys.mosaiq.execute(
        cursor,
        """
        SELECT
            Ident.IDA,
            Patient.Last_Name,
            Patient.First_Name,
            Chklist.Due_DtTm,
            Chklist.Instructions,
            Chklist.Notes,
            QCLTask.Description
        FROM Chklist, Staff, QCLTask, Ident, Patient
        WHERE
            Chklist.Pat_ID1 = Ident.Pat_ID1 AND
            Patient.Pat_ID1 = Ident.Pat_ID1 AND
            QCLTask.TSK_ID = Chklist.TSK_ID AND
            Staff.Staff_ID = Chklist.Rsp_Staff_ID AND
            Staff.Last_Name = %(location)s AND
            Chklist.Complete = 0
        """,
        {"location": location},
    )

    results = pd.DataFrame(
        data=data,
        columns=[
            "patient_id",
            "last_name",
            "first_name",
            "due",
            "instructions",
            "comment",
            "task",
        ],
    )

    results = results.sort_values(by=["due"], ascending=False)

    return results

In [ ]:
with pymedphys.mosaiq.connect('msqsql') as cursor:
    qcls = get_incomplete_qcls(cursor, 'Physics_Check')

In [ ]:
qcls

In [ ]:
patient_ids = qcls['patient_id']

In [ ]:
logfile_paths = {}

for patient_id in patient_ids:
    paths = list(logfile_directory.joinpath('indexed').glob(f'*/{patient_id}_*/*/*/*/*.trf'))
    if paths:
        logfile_paths[patient_id] = paths

In [ ]:
logfile_paths

In [ ]:
patiend_ids_with_logfiles = list(logfile_paths.keys())

In [ ]:
patient_id = patiend_ids_with_logfiles[0]

In [ ]:
monaco_approval_comments = {}

for patient_id in patiend_ids_with_logfiles:
    paths = list(monaco_directory.glob(f'*~{patient_id}/plan/*/*plan_comment'))
    if paths:
        monaco_approval_comments[patient_id] = paths

In [ ]:
monaco_approval_comments

In [ ]:
patient_ids_with_approved_plans = list(monaco_approval_comments.keys())

In [ ]:
tel_files = {}

for patient_id in patient_ids_with_approved_plans:
    paths = list(monaco_directory.glob(f'*~{patient_id}/plan/*/tel.1'))
    if paths:
        tel_files[patient_id] = paths

In [ ]:
tel_files

In [ ]:
patient_ids_to_check = list(tel_files.keys())

In [ ]:
# data_paths = list(pathlib.Path(r'C:\Users\sbiggs\Desktop\20191223_logfile_compare').glob('*/*'))
# data_paths

In [ ]:
# def get_file_type(input_paths, file_type):
#     paths = [path for path in input_paths if file_type in path.name]
#     assert len(paths) == 1
#     return paths[0]

In [ ]:
def plot_gamma_hist(gamma, percent, dist):
    valid_gamma = gamma[~np.isnan(gamma)]

    plt.hist(valid_gamma, 50, density=True)
    pass_ratio = np.sum(valid_gamma <= 1) / len(valid_gamma)

    plt.title("Local Gamma ({0}%/{1}mm) | Percent Pass: {2:.2f} % | Max Gamma: {3:.2f}".format(percent, dist, pass_ratio*100, np.max(valid_gamma)))

In [ ]:
def markdown_print(string):
    IPython.display.display(IPython.display.Markdown(string))

In [ ]:
grid = pymedphys.mudensity.grid()

def plot_and_save_results(mudensity_tel, mudensity_trf, gamma, output_dir):
    plt.figure()
    pymedphys.mudensity.display(grid, mudensity_tel)
    plt.title("Monaco Plan")
    plt.savefig(output_dir.joinpath("001_monaco.png"))

    plt.figure()
    pymedphys.mudensity.display(grid, mudensity_trf)
    plt.title("Logfile Result")
    plt.savefig(output_dir.joinpath("002_logfile.png"))

    diff = mudensity_trf - mudensity_tel
    largest_item = np.max(np.abs(diff))

    plt.figure()
    pymedphys.mudensity.display(grid, diff, cmap="seismic", vmin=-largest_item, vmax=largest_item)
    plt.title("Logfile - Monaco")
    plt.savefig(output_dir.joinpath("003_diff.png"))

    plt.figure()
    pymedphys.mudensity.display(grid, gamma, cmap="coolwarm", vmin=0, vmax=2)
    plt.title(f"Local Gamma | {percent_deviation}%/{mm_dist_threshold}mm")
    plt.savefig(output_dir.joinpath("004_gamma_map.png"))

    plt.figure()
    plot_gamma_hist(gamma, percent_deviation, mm_dist_threshold)
    plt.savefig(output_dir.joinpath("005_gamma_hist.png"))

    plt.show()

In [ ]:
@functools.lru_cache()
def get_delivery_tel_file(filepath):
    delivery_tel = pymedphys.Delivery.from_monaco(filepath)
    
    return delivery_tel


@functools.lru_cache()
def get_delivery_trf_file(filepath):
    delivery_trf = pymedphys.Delivery.from_logfile(filepath)
    
    return delivery_trf

In [ ]:
@functools.lru_cache()
def get_mu_density_from_file(filepath):
    if filepath.suffix == '.trf':
        delivery = get_delivery_trf_file(filepath)
    elif filepath.name == 'tel.1':
        delivery = pymedphys.Delivery.from_monaco(filepath)
    else:
        raise ValueError("Not appropriate file type found")
        
    mudensity = delivery.mudensity()
    
    return mudensity

In [ ]:
coords = (grid['jaw'], grid['mlc'])

@functools.lru_cache()
def calc_gamma(mudensity_tel, mudensity_trf):
    gamma = pymedphys.gamma(
        coords, mudensity_tel, coords, mudensity_trf, percent_deviation, mm_dist_threshold, 
        local_gamma=True, quiet=True
    )
    
    return gamma

In [ ]:
def to_tuple(array):
    return tuple(map(tuple, array))

# coords

In [ ]:
def run_for_a_patient_id(patient_id):
    markdown_print(f"# {patient_id}")
    
    for tel_file in tel_files[patient_id]:
        markdown_print(f"## `{tel_file}`")
        
        try:
            mudensity_tel = get_mu_density_from_file(tel_file)

            for trf_file in logfile_paths[patient_id]:
                markdown_print(f"### `{trf_file}`")

                mudensity_trf = get_mu_density_from_file(trf_file)

                gamma = calc_gamma(to_tuple(mudensity_tel), to_tuple(mudensity_trf))

                results_dir = output_directory.joinpath(patient_id, tel_file.parent.name, trf_file.stem)
                results_dir.mkdir(exist_ok=True, parents=True)
                plot_and_save_results(mudensity_tel, mudensity_trf, gamma, results_dir)
        
        except Exception as e:
            traceback.print_exc()


for patient_id in patient_ids_to_check:
    run_for_a_patient_id(patient_id)